In [1]:
import cv2
import numpy as np
import random

def center_img(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    out = np.zeros_like(gray, dtype=np.uint32)

    # Apply thresholding
    _, thresh = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Calculate center of each contour
    centers = []
    for i, cnt in enumerate(contours):
        M = cv2.moments(cnt)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            # print(cx, cy)
            # img = cv2.circle(img, (cx, cy), 15, (0, 255, 0), 10)
            centers.append((cx, cy))
            color = random.randint(0, 255)
            # while color in done:
            #     color = random.randint(0, 255)
            # done.add(color)
            gray = cv2.drawContours(gray, contours, i, color=30, thickness=-1)
            out[gray == 30] = i+1
            gray[gray == 30] = 1
            assert not np.count_nonzero(gray == 30)
    return centers, out


def entity_recall(gt_contour, pred):
    contours = np.unique(gt_contour)
    contours = contours[contours != 0]
    num_right = 0
    for i , cnt in enumerate(contours):
        num = np.count_nonzero(gt_contour == cnt)
        num_intersect = np.count_nonzero(pred[gt_contour == cnt] == 255)
        if num_intersect/num >= 0.5: 
            num_right +=1
    return num_right/len(contours)

In [ ]:
# slices 100-110
